In [1]:
import sys
sys.path.append('/home/ivan/distribution_connector')
import os

In [2]:
from sklearn import datasets
import numpy as np
import torch
from tqdm import tqdm
from connector_utils import test_models, gather_statistics, test_func
import matplotlib.pyplot as plt

In [3]:
from connector import Connector
# from one_layer_utils import samples, make_dataset, get_model, get_b
from utils import test_model
from flows.models.utils import test_flow

# MNIST

In [4]:
import models
architecture = getattr(models, "LinearOneLayer") #LinearOneLayer LogRegression

import data
loaders, num_classes = data.loaders(
    "MNIST",
    "data",
    1024,
    1,
    "VGG",
    True)

model1 = architecture.base(num_classes=10, **architecture.kwargs)
model2 = architecture.base(num_classes=10, **architecture.kwargs)

model1.load_state_dict(torch.load('curves_mnist/LinearOneLayer/LongTraining/curve16/checkpoint-30.pt')['model_state'])
model2.load_state_dict(torch.load('curves_mnist/LinearOneLayer/LongTraining/curve17/checkpoint-30.pt')['model_state'])

import pickle
data_path = '/home/ivan/dnn-mode-connectivity/data/MNIST.pickle'
with open(data_path, 'rb') as handle:
    dataset, B = pickle.load(handle)
b2 = torch.FloatTensor([np.array(B).mean(0)])
b = np.array(B).mean(0)

train_dataset, test_dataset = dataset[:-8000], dataset[-8000:]
modes = [dataset[-4000:-2000], dataset[-2000:]]

You are going to run models on the test set. Are you sure?


In [5]:
def samples(model):
    p = [list(model.parameters())[i].data.cpu().numpy() for i in range(len(list(model.parameters())))]
    return p

def samples_per_layer(model, bias=True):
    p = samples(model)
    if bias:
        p = [np.hstack([p[i], p[i+1][:, None]]) for i in range(0, len(p), 2)]        
    return p

def samples_butterfly(model, bias=True):
    if bias:
        return None
    else:   
        p = samples_per_layer(model, bias=bias)
        parameters = [np.hstack([p[i], p[i+1].T]) for i in range(0, len(p), 2)]
    return parameters

def get_model(W, architecture, bias=False, per_layer=True):
    model_sampled = architecture.base(num_classes=10, **architecture.kwargs)
    model_samples = np.array(W)  

    if per_layer:
        for parameter, w in zip(model_sampled.parameters(), W):
            parameter.data.copy_(torch.from_numpy(w))
    else:
        for i, parameter in enumerate(model_sampled.parameters()):
            w = W[i//2]
            if i % 2 == 0:
                offset = 0
            N = parameter.data.shape[1]
            w_part = w[:, offset:offset+N]
            offset = N
            if i % 2 == 0:
                parameter.data.copy_(torch.from_numpy(w_part))
            else:
                parameter.data.copy_(torch.from_numpy(w_part.T))
            

    return model_sampled

def get_model(W, B, architecture):
    model_sampled = architecture.base(num_classes=10, **architecture.kwargs)
    model_samples = np.array(W)  # .cpu().data.numpy()
    SIZE = model_sampled.middle_dim

    offset = 0
    for parameter in list(model_sampled.parameters())[:-1]:
        size = int(np.prod(parameter.size()) / SIZE)
        value = model_samples[:, offset:offset + size]
        if size == 10 or size == 1:
            value = value.T
        value = value.reshape(parameter.size())
        parameter.data.copy_(torch.from_numpy(value))
        offset += size

    list(model_sampled.parameters())[-1].data.copy_(B.mean(0))  # torch.from_numpy(

    return model_sampled

def samples(model):
    p1 = list(model.parameters())[0].data.cpu().numpy()
    p2 = list(model.parameters())[1].data.cpu().numpy()
    p3 = list(model.parameters())[2].transpose(0, 1).data.cpu().numpy()
    samples = np.hstack([p1, p2[:, None], p3])
    return samples

def transform(x1, x2, E12, E22_inv):
#     print(x1.shape, x2.shape, E12.shape, E22_inv.shape)
    y1 = x1 - E12 @ E22_inv @ x2
    return np.concatenate([y1, x2]).T
   
def inv_transform(y1, y2, E12, E22_inv):
#     print(x1.shape, x2.shape, E12.shape, E22_inv.shape)
    x1 = y1 + E12 @ E22_inv @ y2
    return np.concatenate([x1, y2]).T

In [6]:
parameters1 = samples(model1)
parameters2 = samples(model2)

parameters = np.concatenate([parameters1, parameters2])

In [15]:
def connect_simple(W1, W2, lines=True, func='arc_connect', mean=0, compute=True):
    
        
    W1 = W1.T
    W2 = W2.T
    if compute:
        W = np.vstack([W1, W2])
        print('W', W.shape)
        mean = W.mean(0)
        
    W1 -= mean
    W2 -= mean  
    W1 = W1.T
    W2 = W2.T
    
    if lines:    
        print('W1', W1.shape)
        cntr = Connector(W1, W2)
        f = getattr(cntr, func)
        p_res = f()[1]
    else:
        print('W1', W1.T.shape)
        cntr = Connector(W1.T, W2.T)
        f = getattr(cntr, func)
        p_res = f()[1].T
    
    p_res = p_res.T
    p_res += mean
    p_res = p_res.T
    
    return p_res

In [16]:
param1 = [parameters1[:, :-10].T, parameters1[:, -10:]]
param2 = [parameters2[:, :-10].T, parameters2[:, -10:]]

In [17]:
param1[0].shape, param1[1].shape

((785, 2000), (2000, 10))

In [21]:
pres = []
pres.append(connect_simple(param1[0], param2[0], lines=False, func='arc_connect_PCA', compute=True))
pres.append(connect_simple(param1[1], param2[1], lines=True, func='arc_connect_PCA',  compute=False))
p_res = np.hstack([pres[0].T, pres[1]])
model = get_model(p_res, b2, architecture)
model.cuda();
model.eval();
test_model(model, loaders, cuda=True)

W (4000, 785)
W1 (2000, 785)
W1 (2000, 10)
train results {'nll': 0.08186667322715123, 'loss': 0.08186667322715123, 'accuracy': 98.15166666666667}
test results {'nll': 0.10232745043039322, 'loss': 0.10232745043039322, 'accuracy': 97.37}


({'nll': 0.08186667322715123,
  'loss': 0.08186667322715123,
  'accuracy': 98.15166666666667},
 {'nll': 0.10232745043039322, 'loss': 0.10232745043039322, 'accuracy': 97.37})

In [ ]:


funcs = ['arc_connect']

for func in funcs:
    parameters_res = []
    print(func)
    for p1, p2 in zip(param1, param2):
        cntr = Connector(p1, p2)
        print(p1.shape)
        f = getattr(cntr, func)
        if 'PCA' in func:
            K = min(300, p1.shape[0], p1.shape[1])
            res = f(K=K)[1]
        elif 'third_cumulant' in func:
            K = min(100, p1.shape[0], p1.shape[1])
            res = f(K=K)[1] 
        else:
            res = f()[1]
            
        print('res', res.shape)
        parameters_res.append(res)
        
    parameters_res = np.hstack([parameters_res[0].T, parameters_res[1]])
    model = get_model(parameters_res, b2, architecture)
    model.cuda();
    model.eval();
    test_model(model, loaders, cuda=True)

In [10]:
func = 'arc_connect'
print(func)
cntr = Connector(parameters1_tr, parameters2_tr)
f = getattr(cntr, func)
parameters_res = f()[1]

arc_connect


In [149]:
parameters_res_tr = inv_transform(parameters_res[:, :-10].T, parameters_res[:, -10:].T, E12,  E22_inv)

In [151]:
model = get_model(parameters_res_tr, b2, architecture)
model.cuda();
model.eval();
test_model(model, loaders, cuda=True)

train results {'nll': 0.07982834839820861, 'loss': 0.07982834839820861, 'accuracy': 97.99666666666667}
test results {'nll': 0.10293639920949936, 'loss': 0.10293639920949936, 'accuracy': 97.14}


({'nll': 0.07982834839820861,
  'loss': 0.07982834839820861,
  'accuracy': 97.99666666666667},
 {'nll': 0.10293639920949936, 'loss': 0.10293639920949936, 'accuracy': 97.14})

In [79]:
# parameters_res_tr.shape

In [33]:
param_new = parameters1[:3]+parameters2[3:]

In [34]:
model = get_model(param_new, architecture, per_layer=True)
model.cuda();
model.eval();
test_model(model, loaders, cuda=True)

In [ ]:
for func in ['lin_connect', 'arc_connect', 'arc_connect_PCA']:
    parameters_res = []
    print(func)
    for p1, p2 in zip(parameters1, parameters2):
        cntr = Connector(p1, p2)
        f = getattr(cntr, func)
        if 'PCA' in func:
            K = min(300, p1.shape[0], p1.shape[1])
            res = f(K=K)[1]
        elif 'third_cumulant' in func:
            K = min(100, p1.shape[0], p1.shape[1])
            res = f(K=K)[1] 
        else:
            res = f()[1]
        parameters_res.append(res)

    model = get_model(parameters_res, architecture, per_layer=True)
    model.cuda();
    model.eval();
    test_model(model, loaders, cuda=True)

# CIFAR10

In [40]:
import models
architecture = getattr(models, "LinearOneLayerCF") #LinearOneLayer LogRegression

import data
loaders, num_classes = data.loaders(
    "CIFAR10",
    "data",
    1024,
    1,
    "VGG",
    False)

model1 = architecture.base(num_classes=10, **architecture.kwargs)
model2 = architecture.base(num_classes=10, **architecture.kwargs)

model1.load_state_dict(torch.load('curves/LinearOneLayer/curve16/checkpoint-400.pt')['model_state'])
model2.load_state_dict(torch.load('curves/LinearOneLayer/curve17/checkpoint-400.pt')['model_state'])

import pickle
data_path = '/home/ivan/dnn-mode-connectivity/data/CIFAR.pickle'
with open(data_path, 'rb') as handle:
    dataset, B = pickle.load(handle)
b2 =  torch.FloatTensor([np.array(B).mean(0)])
b = np.array(B).mean(0)

train_dataset, test_dataset = dataset[:-8000], dataset[-8000:]
modes = [dataset[-4000:-2000], dataset[-2000:]]

Files already downloaded and verified
You are going to run models on the test set. Are you sure?
Files already downloaded and verified


In [41]:
parameters1 = samples(model1)
parameters2 = samples(model2)

parameters = np.concatenate([parameters1, parameters2])

In [42]:
param1 = [parameters1[:, :-10].T, parameters1[:, -10:]]
param2 = [parameters2[:, :-10].T, parameters2[:, -10:]]

In [43]:
pres = []
pres.append(connect_simple(param1[0], param2[0], lines=False, func='arc_connect', compute=False))
pres.append(connect_simple(param1[1], param2[1], lines=True, func='arc_connect',  compute=False))
p_res = np.hstack([pres[0].T, pres[1]])
model = get_model(p_res, b2, architecture)
model.cuda();
model.eval();
test_model(model, loaders, cuda=True)

W1 (2000, 3073)
W1 (2000, 10)
train results {'nll': 1.4840123331832886, 'loss': 1.4840123331832886, 'accuracy': 51.31}
test results {'nll': 2.6923051845550536, 'loss': 2.6923051845550536, 'accuracy': 40.93}


({'nll': 1.4840123331832886, 'loss': 1.4840123331832886, 'accuracy': 51.31},
 {'nll': 2.6923051845550536, 'loss': 2.6923051845550536, 'accuracy': 40.93})

In [23]:
param1 = [parameters1[:, :-10], parameters1[:, -10:]]
param2 = [parameters2[:, :-10], parameters2[:, -10:]]

for func in ['lin_connect', 'arc_connect', 'arc_connect_PCA']:
    parameters_res = []
    print(func)
    for p1, p2 in zip(param1, param2):
        cntr = Connector(p1, p2)
        print(p1.shape)
        f = getattr(cntr, func)
        if 'PCA' in func:
            K = min(300, p1.shape[0], p1.shape[1])
            res = f(K=K)[1]
        elif 'third_cumulant' in func:
            K = min(100, p1.shape[0], p1.shape[1])
            res = f(K=K)[1] 
        else:
            res = f()[1]
            
        print('res', res.shape)
        parameters_res.append(res)
        
    parameters_res = np.hstack([parameters_res[0], parameters_res[1]])
    model = get_model(parameters_res, b2, architecture)
    model.cuda();
    model.eval();
    test_model(model, loaders, cuda=True)

lin_connect
(2000, 3073)
res (2000, 3073)
(2000, 10)
res (2000, 10)
train results {'nll': 1.8912518212890626, 'loss': 1.8912518212890626, 'accuracy': 32.088}
test results {'nll': 2.0686955657958985, 'loss': 2.0686955657958985, 'accuracy': 37.79}
arc_connect
(2000, 3073)
res (2000, 3073)
(2000, 10)
res (2000, 10)
train results {'nll': 1.4912290412139892, 'loss': 1.4912290412139892, 'accuracy': 50.816}
test results {'nll': 2.6923051845550536, 'loss': 2.6923051845550536, 'accuracy': 40.93}
arc_connect_PCA
(2000, 3073)
res (2000, 3073)
(2000, 10)
res (2000, 10)
train results {'nll': 1.4851501541519165, 'loss': 1.4851501541519165, 'accuracy': 51.288}
test results {'nll': 2.8195617031097413, 'loss': 2.8195617031097413, 'accuracy': 40.16}


In [15]:
param1 = [parameters1[:, :-10].T, parameters1[:, -10:]]
param2 = [parameters2[:, :-10].T, parameters2[:, -10:]]

for func in ['lin_connect', 'arc_connect', 'arc_connect_PCA']:
    parameters_res = []
    print(func)
    for p1, p2 in zip(param1, param2):
        cntr = Connector(p1, p2)
        print(p1.shape)
        f = getattr(cntr, func)
        if 'PCA' in func:
            K = min(300, p1.shape[0], p1.shape[1])
            res = f(K=K)[1]
        elif 'third_cumulant' in func:
            K = min(100, p1.shape[0], p1.shape[1])
            res = f(K=K)[1] 
        else:
            res = f()[1]
            
        print('res', res.shape)
        parameters_res.append(res)
        
    parameters_res = np.hstack([parameters_res[0].T, parameters_res[1]])
    model = get_model(parameters_res, b2, architecture)
    model.cuda();
    model.eval();
    test_model(model, loaders, cuda=True)

lin_connect
(3073, 2000)
res (3073, 2000)
(2000, 10)
res (2000, 10)
train results {'nll': 1.8950114845657349, 'loss': 1.8950114845657349, 'accuracy': 32.046}
test results {'nll': 2.0686955657958985, 'loss': 2.0686955657958985, 'accuracy': 37.79}
arc_connect
(3073, 2000)
res (3073, 2000)
(2000, 10)
res (2000, 10)
train results {'nll': 1.6132043946456909, 'loss': 1.6132043946456909, 'accuracy': 43.942}
test results {'nll': 2.6095668628692628, 'loss': 2.6095668628692628, 'accuracy': 41.38}
arc_connect_PCA
(3073, 2000)
res (3073, 2000)
(2000, 10)
res (2000, 10)
train results {'nll': 1.6867427646255493, 'loss': 1.6867427646255493, 'accuracy': 40.956}
test results {'nll': 2.651808978652954, 'loss': 2.651808978652954, 'accuracy': 40.74}


In [25]:
param1 = [parameters1[:, :-10].T, parameters1[:, -10:].T]
param2 = [parameters2[:, :-10].T, parameters2[:, -10:].T]

for func in ['lin_connect', 'arc_connect', 'arc_connect_PCA']:
    parameters_res = []
    print(func)
    for p1, p2 in zip(param1, param2):
        cntr = Connector(p1, p2)
        print(p1.shape)
        f = getattr(cntr, func)
        if 'PCA' in func:
            K = min(300, p1.shape[0], p1.shape[1])
            res = f(K=K)[1]
        elif 'third_cumulant' in func:
            K = min(100, p1.shape[0], p1.shape[1])
            res = f(K=K)[1] 
        else:
            res = f()[1]
            
        print('res', res.shape)
        parameters_res.append(res)
        
    parameters_res = np.hstack([parameters_res[0].T, parameters_res[1].T])
    model = get_model(parameters_res, b2, architecture)
    model.cuda();
    model.eval();
    test_model(model, loaders, cuda=True)

lin_connect
(3073, 2000)
res (3073, 2000)
(10, 2000)
res (10, 2000)
train results {'nll': 1.894404046974182, 'loss': 1.894404046974182, 'accuracy': 32.104}
test results {'nll': 2.0686955657958985, 'loss': 2.0686955657958985, 'accuracy': 37.79}
arc_connect
(3073, 2000)
res (3073, 2000)
(10, 2000)
res (10, 2000)
train results {'nll': 1.5941276955795287, 'loss': 1.5941276955795287, 'accuracy': 44.44}
test results {'nll': 2.962168966674805, 'loss': 2.962168966674805, 'accuracy': 41.61}
arc_connect_PCA
(3073, 2000)
res (3073, 2000)
(10, 2000)
res (10, 2000)
train results {'nll': 1.7525887490463257, 'loss': 1.7525887490463257, 'accuracy': 37.204}
test results {'nll': 2.861420792388916, 'loss': 2.861420792388916, 'accuracy': 37.24}


In [14]:
model1.cuda();
model1.eval();
test_model(model1, loaders, cuda=True)

train results {'nll': 0.8493014687347412, 'loss': 0.8493014687347412, 'accuracy': 70.424}
test results {'nll': 1.313883560180664, 'loss': 1.313883560180664, 'accuracy': 58.9}


({'nll': 0.8493014687347412, 'loss': 0.8493014687347412, 'accuracy': 70.424},
 {'nll': 1.313883560180664, 'loss': 1.313883560180664, 'accuracy': 58.9})

In [120]:
func = 'arc_connect'
parameters_res = []
print(func)
cntr = Connector(parameters1_tr, parameters2_tr)
f = getattr(cntr, func)
parameters_res = f()[1]

arc_connect


In [121]:
# parameters_res.shape

In [122]:
parameters_res_tr = inv_transform(parameters_res[:, :-10].T, parameters_res[:, -10:].T, E12,  E22_inv)

In [123]:
model = get_model(parameters_res_tr, b2, architecture)
model.cuda();
model.eval();
test_model(model, loaders, cuda=True)

train results {'nll': 1.4964728350067138, 'loss': 1.4964728350067138, 'accuracy': 51.208}
test results {'nll': 2.692305062484741, 'loss': 2.692305062484741, 'accuracy': 40.93}


({'nll': 1.4964728350067138, 'loss': 1.4964728350067138, 'accuracy': 51.208},
 {'nll': 2.692305062484741, 'loss': 2.692305062484741, 'accuracy': 40.93})

# Gavering sample and error time plots on CIFAR10

In [5]:
# for func_name in funcs:
#     t = np.linspace(0,1, 61)[1:-1]
#     stat = test_func(modes, loaders, b2, architecture, func_name, flow=None, K=400, t=t, 
#                      show=False, verbose=False, save_pic=True, path='CIFAR', save_samples=False)
#     t = np.linspace(0,1, 5)[1:-1]
#     stat = test_func(modes, loaders, b2, architecture, func_name, flow=None, K=400, t=t, 
#                      show=False, verbose=False, save_pic=False, path='CIFAR', save_samples=True)

## RNVP flow

In [6]:
# t = np.linspace(0,1, 61)[1:-1]
# stat = test_func(modes, loaders, b2, architecture, func_name='flow_connect', flow=flow, K=400, t=t, 
#                  show=False, verbose=False, save_pic=True, test_models=True, path='CIFAR', save_samples=False)

In [83]:
# t = np.linspace(0,1, 5)[1:-1]
# stat = test_func(modes, loaders, b2, architecture, func_name='flow_connect', flow=flow, K=400, t=t, 
#                  show=False, verbose=False, save_pic=False, path='CIFAR', save_samples=True)

# Computing mean and std for different methods on CIFAR10

In [14]:
model1.cuda()
test_model(model1, loaders, cuda=True)

train results {'nll': 0.8489676392555237, 'loss': 0.8489676392555237, 'accuracy': 70.572}
test results {'nll': 1.313883560180664, 'loss': 1.313883560180664, 'accuracy': 58.9}


({'nll': 0.8489676392555237, 'loss': 0.8489676392555237, 'accuracy': 70.572},
 {'nll': 1.313883560180664, 'loss': 1.313883560180664, 'accuracy': 58.9})

## flow

In [87]:
from flows.models import RealNVP, RealNVP_OneLayer_full, IAF_OneLayer_full

flow = RealNVP_OneLayer_full(in_dim=3083, dim_middle=3000, N_layers=1, batch_norm=False, data_b2=b)
flow.load_state_dict(torch.load('/home/ivan/distribution_connector/flow_models/RealNVP/CIFAR_test1/checkpoint-7.pt')['model_state'])


mask len 2


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [88]:
# from flows.models import RealNVP, RealNVP_OneLayer_full, IAF_OneLayer_full
# flow = IAF_OneLayer_full(in_dim=3083, dim_middle=3000, N_layers=1, batch_norm=False, data_b2=b)
# flow.load_state_dict(torch.load('/home/ivan/distribution_connector/flow_models/RealNVP/CIFAR_test1/checkpoint-7.pt')['model_state'])

In [89]:
flow.cuda();
stat_dict, stat_dict_mean, stat_dict_std = gather_statistics(dataset[-8000:], 
                                                             loaders, b2, architecture, 
                                                             flow=flow, K=80, t=0.5, 
                                                             verbose=True, func_name=['flow_connect'],
                                                             save_stat=True, path='stat/CIFAR/bijection')



  0%|          | 0/4 [00:00<?, ?it/s]

N_models 4
flow_connect
0.5
train results {'nll': 1.0533692544174194, 'loss': 1.0533692544174194, 'accuracy': 63.456}
test results {'nll': 1.6187260831832886, 'loss': 1.6187260831832886, 'accuracy': 53.59}
flow_connect
0.5
train results {'nll': 1.0545070586013794, 'loss': 1.0545070586013794, 'accuracy': 63.1}
test results {'nll': 1.7111023202896118, 'loss': 1.7111023202896118, 'accuracy': 52.64}
flow_connect
0.5




 25%|██▌       | 1/4 [01:00<03:01, 60.47s/it]

train results {'nll': 1.0488071635627747, 'loss': 1.0488071635627747, 'accuracy': 63.788}
test results {'nll': 1.5592954841613769, 'loss': 1.5592954841613769, 'accuracy': 55.26}
flow_connect
0.5
train results {'nll': 1.0633495126342773, 'loss': 1.0633495126342773, 'accuracy': 63.462}
test results {'nll': 1.5593252670288087, 'loss': 1.5593252670288087, 'accuracy': 54.64}
flow_connect
0.5




 50%|█████     | 2/4 [01:41<01:49, 54.79s/it]

train results {'nll': 1.0475879463577271, 'loss': 1.0475879463577271, 'accuracy': 63.818}
test results {'nll': 1.5472380882263184, 'loss': 1.5472380882263184, 'accuracy': 54.58}
flow_connect
0.5




 75%|███████▌  | 3/4 [02:03<00:44, 44.90s/it]

100%|██████████| 4/4 [02:03<00:00, 30.96s/it]

train results {'nll': 1.0577251321029664, 'loss': 1.0577251321029664, 'accuracy': 63.144}
test results {'nll': 1.6692164546966552, 'loss': 1.6692164546966552, 'accuracy': 52.93}
saving to  data/stat/CIFAR/bijectionerror_stat.pickle


## basic 

In [7]:
test_dataset = dataset[-8000:]

In [11]:
funcs = [func for func in dir(cntr) if callable(getattr(cntr, func)) and 'connect' in func 
         and 'flow' not in func and 'third_cumulant' in func]

In [12]:
funcs

['third_cumulant_connect']

In [14]:
stat_dict, stat_dict_mean, stat_dict_std = gather_statistics(test_dataset, 
                                                             loaders, b2, architecture, 
                                                             flow=None, K=200, t=0.5, 
                                                             verbose=True, func_name=funcs,
                                                             save_stat=True, path='stat/CIFAR/third_cumulant')


  0%|          | 0/4 [00:00<?, ?it/s]

N_models 4
third_cumulant_connect
c 49339.753945420765 8000000
0.5
train results {'nll': 192.03328536132813, 'loss': 192.03328536132813, 'accuracy': 10.95}
test results {'nll': 159.18884404296875, 'loss': 159.18884404296875, 'accuracy': 11.4}
third_cumulant_connect
c 49147.18525962689 8000000
0.5
train results {'nll': 197.92421240234376, 'loss': 197.92421240234376, 'accuracy': 10.81}
test results {'nll': 172.68769267578125, 'loss': 172.68769267578125, 'accuracy': 11.54}
third_cumulant_connect
c 49338.789716830164 8000000
0.5



 25%|██▌       | 1/4 [19:02<57:07, 1142.60s/it]

train results {'nll': 200.2789377734375, 'loss': 200.2789377734375, 'accuracy': 11.184}
test results {'nll': 174.5269504638672, 'loss': 174.5269504638672, 'accuracy': 12.38}
third_cumulant_connect
c 49031.59461684667 8000000
0.5
train results {'nll': 196.78189912109374, 'loss': 196.78189912109374, 'accuracy': 10.758}
test results {'nll': 166.34349104003905, 'loss': 166.34349104003905, 'accuracy': 11.96}
third_cumulant_connect
c 49216.40067193843 8000000
0.5



 50%|█████     | 2/4 [32:55<34:59, 1049.84s/it]

train results {'nll': 203.76456399902344, 'loss': 203.76456399902344, 'accuracy': 10.178}
test results {'nll': 165.93680776367188, 'loss': 165.93680776367188, 'accuracy': 11.31}
third_cumulant_connect
c 49634.76412962114 8000000
0.5



100%|██████████| 4/4 [39:04<00:00, 586.23s/it]

train results {'nll': 204.2034564892578, 'loss': 204.2034564892578, 'accuracy': 10.938}
test results {'nll': 168.1272219970703, 'loss': 168.1272219970703, 'accuracy': 12.78}
saving to  data/stat/CIFAR/third_cumulanterror_stat.pickle


# Checking

In [23]:
with open('data/stat/CIFAR/third_cumulanterror_stat.pickle', 'rb') as handle:
        load = pickle.load(handle)

In [24]:
load[1]

{'train': {'third_cumulant_connect': 10.802999999999999},
 'test': {'third_cumulant_connect': 11.895000000000001}}

In [25]:
load[2]

{'train': {'third_cumulant_connect': 0.31029394665918486},
 'test': {'third_cumulant_connect': 0.5378893938348291}}

In [26]:
stat_dict_mean, stat_dict_std

({'train': {'flow_connect': 63.60966666666667},
  'test': {'flow_connect': 53.94}},
 {'train': {'flow_connect': 0.353156087618808},
  'test': {'flow_connect': 0.9552137631615929}})